# Tensorflow 튜토리얼
## 소개
텐서플로우(TensorFlow)는 기계 학습과 딥러닝을 위해 구글에서 만든 오픈소스 라이브러리입니다. 데이터 플로우 그래프(Data Flow Graph) 방식을 사용하였습니다.

### 데이터 플로우 그래프

데이터 플로우 그래프는 수학 계산과 데이터의 흐름을 노드(Node)와 엣지(Edge)를 사용한 방향 그래프(Directed Graph)로 표현합니다.

![data flow graph](https://www.tensorflow.org/images/tensors_flowing.gif)

노드는 수학적 계산, 데이터 입/출력, 그리고 데이터의 읽기/저장 등의 작업을 수행합니다. 엣지는 노드들 간 데이터의 입출력 관계를 나타냅니다.

엣지는 동적 사이즈의 다차원 데이터 배열(=텐서)을 실어나르는데, 여기에서 텐서플로우라는 이름이 지어졌습니다.

> 텐서(Tensor)는 과학과 공학 등 다양한 분야에서 이전부터 쓰이던 개념입니다. 수학에서는 [임의의 기하 구조를 좌표 독립적으로 표현](http://ghebook.blogspot.kr/2011/06/tensor.html)하기 위한 표기법으로 알려져 있지만, 분야마다 조금씩 다른 의미로 사용됩니다. 여기에서는 *학습 데이터가 저장되는 다차원 배열* 정도로 이해하시면 되겠습니다.

### 특징

텐서플로우는 다음과 같은 특징을 가집니다:

- 데이터 플로우 그래프를 통한 풍부한 표현력
- 코드 수정 없이 CPU/GPU 모드로 동작
- 아이디어 테스트에서 서비스 단계까지 이용 가능
- 계산 구조와 목표 함수만 정의하면 자동으로 미분 계산을 처리 
- Python/C++를 지원하며, [SWIG](http://www.swig.org)를 통해 다양한 언어 지원 가능

*이후의 설명은 [Python](https://www.python.org)을 중심으로 진행하겠습니다.* (pip를 통한 Python3설치는 개발 중으로, Python2 기반으로 하겠습니다.)

> *"구글이 텐서플로우를 오픈소스로 한 것은, 기계 학습이 앞으로 제품과 기술을 혁신하는데 가장 필수적인 요소라고 믿기 때문입니다."      - Google Brain Team*


## 기본 개념 익히기

일단 기본 용어부터 살펴보겠습니다.

### 용어
#### 오퍼레이션(Operation)
그래프 상의 노드는 오퍼레이션(줄임말 *op*)으로 불립니다. 오퍼레이션은 하나 이상의 *텐서*를 받을 수 있습니다. 오퍼레이션은 계산을 수행하고, 결과를 하나 이상의 텐서로 반환할 수 있습니다.

#### 텐서(Tensor)
내부적으로 모든 데이터는 텐서를 통해 표현됩니다. 텐서는 일종의 다차원 배열인데, 그래프 내의 오퍼레이션 간에는 텐서만이 전달됩니다. ([Caffe](http://caffe.berkeleyvision.org)의 [Blob](http://caffe.berkeleyvision.org/tutorial/net_layer_blob.html)과 유사합니다.)

#### 세션(Session)

그래프를 실행하기 위해서는 [세션](https://www.tensorflow.org/versions/master/api_docs/python/client.html#session-management) 객체가 필요합니다. 세션은 오퍼레이션의 실행 환경을 캡슐화한 것입니다.

#### 변수(Variables)
[변수](https://www.tensorflow.org/versions/master/how_tos/variables/index.html)는 그래프의 실행시, 패러미터를 저장하고 갱신하는데 사용됩니다. 메모리 상에서 텐서를 저장하는 버퍼 역할을 합니다.


## Single-layer perceptron으로 XOR구현하기

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_data = np.array([[0], [1], [1], [0]])

X = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
Y = tf.placeholder(tf.float32, shape=[4,1], name="y-input")

W = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0))

# hypothesis
h = tf.matmul(X, W)
hypothesis = tf.div(1., 1.+tf.exp(-h))
#hypothesis = tf.sigmoid(tf.matmul(X, W))

# cost function
cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))

# Minimize
a = tf.Variable(0.01) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

# Launch the graph.
with tf.Session() as sess:
    sess.run(init)
    
    # Fit the line.
    for step in range(1000):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}), sess.run(W))
            
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))


0 0.798489 [[ 0.61553425]
 [ 0.92524439]]
200 0.748683 [[ 0.41005871]
 [ 0.68686658]]
400 0.721233 [[ 0.26119283]
 [ 0.50709534]]
600 0.70708 [[ 0.15768373]
 [ 0.37541175]]
800 0.700045 [[ 0.08781303]
 [ 0.28026974]]
[array([[ 0.5       ],
       [ 0.55281162],
       [ 0.51048726],
       [ 0.56315893]], dtype=float32), array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32), array([[False],
       [ True],
       [ True],
       [False]], dtype=bool), 0.5]
Accuracy: 0.5


## Multi-layer perceptron으로 XOR구현하기

In [3]:
x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_data = np.array([[0], [1], [1], [0]])

X = tf.placeholder(tf.float32, shape=[4,2], name="x-input")
Y = tf.placeholder(tf.float32, shape=[4,1], name="y-input")

W1 = tf.Variable(tf.random_uniform([2, 2], -1.0, 1.0), name="Weight1")
W2 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0), name="Weight2")

b1 = tf.Variable(tf.zeros([2]), name="Bias1")
b2 = tf.Variable(tf.zeros([1]), name="Bias2")

# hypothesis
L2 = tf.sigmoid(tf.matmul(X, W1) + b1)
hypothesis = tf.sigmoid(tf.matmul(L2, W2) + b2)

# cost function
#cost = -tf.reduce_mean(Y*tf.log(hypothesis) + (1-Y)*tf.log(1-hypothesis))
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(hypothesis, Y))

# Minimize
a = tf.Variable(0.5) # Learning rate, alpha
optimizer = tf.train.GradientDescentOptimizer(a)
train = optimizer.minimize(cost)

init = tf.initialize_all_variables()

# Launch the graph.
with tf.Session() as sess:
    sess.run(init)
    
    # Fit the line.
    for step in range(20000):
        sess.run(train, feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}),"W1", sess.run(W1),"W2", sess.run(W2))
            
    # Test model
    correct_prediction = tf.equal(tf.floor(hypothesis+0.5), Y)
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    print(sess.run([hypothesis, tf.floor(hypothesis+0.5), correct_prediction, accuracy], feed_dict={X:x_data, Y:y_data}))
    print("Accuracy:", accuracy.eval({X:x_data, Y:y_data}))



0 0.749469 W1 [[ 0.94492376  0.47230342]
 [ 0.78251195 -0.32605469]] W2 [[ 0.82486707]
 [ 0.45189855]]
1000 0.693603 W1 [[ 0.89695239  0.53015965]
 [ 0.71336997 -0.33802629]] W2 [[-0.29987761]
 [-0.71628618]]
2000 0.693309 W1 [[ 0.8590014   0.61026961]
 [ 0.63043636 -0.40766856]] W2 [[-0.31125689]
 [-0.92360377]]
3000 0.693163 W1 [[ 0.83860743  0.7110492 ]
 [ 0.56337982 -0.51218098]] W2 [[-0.28926629]
 [-1.07920134]]
4000 0.693014 W1 [[ 0.8294974   0.8471145 ]
 [ 0.5062573  -0.65339679]] W2 [[-0.24878846]
 [-1.2295239 ]]
5000 0.69276 W1 [[ 0.82819396  1.04076862]
 [ 0.45715648 -0.85128617]] W2 [[-0.17833787]
 [-1.40279412]]
6000 0.69204 W1 [[ 0.83083802  1.35159481]
 [ 0.42461067 -1.16522133]] W2 [[-0.03017333]
 [-1.65099072]]
7000 0.684309 W1 [[ 0.80637026  2.15494728]
 [ 0.57383204 -1.90779757]] W2 [[ 0.55219746]
 [-2.32477593]]
8000 0.616186 W1 [[ 0.82837915  4.54392958]
 [ 1.79427469 -3.79969645]] W2 [[ 2.24434996]
 [-6.0421629 ]]
9000 0.60493 W1 [[ 0.98461968  5.37497854]
 [ 2.194